In [8]:
# Always start clean
import os, shutil

os.chdir("/content")

if os.path.exists("sample_data"):
    shutil.rmtree("sample_data")

if os.path.exists("NeuralNeighborStyleTransfer"):
    shutil.rmtree("NeuralNeighborStyleTransfer")

!git clone https://github.com/nkolkin13/NeuralNeighborStyleTransfer.git
!pip install torch torchvision numpy imageio pillow matplotlib gradio


In [9]:
import os, glob, shutil, subprocess

BASE = "/content/NeuralNeighborStyleTransfer"
INPUT_CONTENT = f"{BASE}/inputs/content"
INPUT_STYLE = f"{BASE}/inputs/style"
OUTPUTS = f"{BASE}/outputs"

os.makedirs(INPUT_CONTENT, exist_ok=True)
os.makedirs(INPUT_STYLE, exist_ok=True)
os.makedirs(OUTPUTS, exist_ok=True)

# Clear default content/style
for folder in [INPUT_CONTENT, INPUT_STYLE]:
    for f in glob.glob(folder + "/*"):
        os.remove(f)

# Download spaghetti style
STYLE_URL = "https://i.imgur.com/9VbJoDy.jpeg"
STYLE_PATH = f"{INPUT_STYLE}/spag.jpeg"

subprocess.run(
    ["curl", "-L", STYLE_URL, "-o", STYLE_PATH],
    check=True
)

print("Environment ready.")


In [10]:
import gradio as gr
import subprocess, os, time
from PIL import Image

STYLE_PATH = "/content/NeuralNeighborStyleTransfer/inputs/style/spag.jpeg"
CONTENT_DIR = "/content/NeuralNeighborStyleTransfer/inputs/content"
OUTPUT_DIR = "/content/NeuralNeighborStyleTransfer/outputs"
SCRIPT_PATH = "/content/NeuralNeighborStyleTransfer/styleTransfer.py"

def run_style_transfer(
    image,
    alpha,
    dont_colorize,
    content_loss,
    high_res,
    no_flip
):
    if image is None:
        return None, None, "No image uploaded."

    # Save uploaded image
    original_name = os.path.basename(image)
    content_path = os.path.join(CONTENT_DIR, original_name)
    Image.open(image).save(content_path)

    name, ext = os.path.splitext(original_name)

    raw_out = f"{OUTPUT_DIR}/spaghetti_{name}_raw.png"
    resized_out = f"{OUTPUT_DIR}/spaghetti_{name}.png"

    cmd = [
        "python", SCRIPT_PATH,
        "--content_path", content_path,
        "--style_path", STYLE_PATH,
        "--output_path", raw_out,
        "--alpha", str(alpha)
    ]

    if dont_colorize:
        cmd.append("--dont_colorize")
    if content_loss:
        cmd.append("--content_loss")
    if high_res:
        cmd.append("--high_res")
    if no_flip:
        cmd.append("--no_flip")

    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            check=True
        )
    except subprocess.CalledProcessError as e:
        return None, None, e.stderr

    # Resize output to original size
    orig = Image.open(content_path)
    out = Image.open(raw_out)
    out_resized = out.resize(orig.size, Image.LANCZOS)
    out_resized.save(resized_out)

    return raw_out, resized_out, result.stdout


with gr.Blocks() as demo:
    gr.Markdown("## Spaghetti Neural Neighbor Style Transfer")

    with gr.Row():
        input_image = gr.Image(type="filepath", label="Upload Image")

    with gr.Row():
        alpha = gr.Slider(0.05, 1.0, value=0.4, label="Alpha: Lower = More spaghetti, Higher = More Original Image")
        dont_colorize = gr.Checkbox(True, label="Preserve Spaghetti colors: Turning this off will make the original image colours come though")
        content_loss = gr.Checkbox(True, label="Enable content loss: Preserves structure more, This creates a shinier look with more highlights for spaghetti")
        high_res = gr.Checkbox(False, label="High resolution (more VRAM): Not typically neccessary")
        no_flip = gr.Checkbox(False, label="Disable patch flipping")

    run_btn = gr.Button("Run Style Transfer")

    with gr.Row():
        raw_out = gr.Image(label="Raw NNST Output")
        final_out = gr.Image(label="Resized to Original Size")

    console = gr.Textbox(label="Console Output", lines=10)

    run_btn.click(
        fn=run_style_transfer,
        inputs=[
            input_image,
            alpha,
            dont_colorize,
            content_loss,
            high_res,
            no_flip
        ],
        outputs=[raw_out, final_out, console]
    )

demo.launch()
